In [1]:
from omegaconf import OmegaConf
import argparse
from models import get_model
from data import get_dataloaders
from optimizers import get_optimizer
from losses import get_loss
from schedulers import get_scheduler
from metrics import get_metrics
from callbacks import get_callbacks
from trainers.base_trainer import BaseTrainer
from utils.logger import setup_logger
from utils.seed import seed_everything
from utils.get_experiment_id import get_experiment_id
from utils.load_checkpoint import load_checkpoint
import torch

In [2]:
config = OmegaConf.load('configs/example.yaml')
config.experiment_id = get_experiment_id(config)
seed_everything(config.training.seed)
logger = setup_logger()
logger.info("Configuración cargada:")
logger.info(OmegaConf.to_yaml(config))

# Configuración cuda                            
# TO DO: Cambiarlo a una forma mas adecuada, y seleccionar la gpu que se quiera usar
if config.training.device == "cuda":
        device = "cuda" if torch.cuda.is_available() else "cpu"
        config.training.device = device
    
# Dataloaders
train_loader, val_loader = get_dataloaders(config)

2025-05-06 01:52:55,143 - Configuración cargada:
2025-05-06 01:52:55,148 - dataset:
  name: CIFAR10
  root: ./data/datasets
  batch_size: 256
  num_workers: 4
model:
  name: resnet18
  weights: ResNet18_Weights.DEFAULT
  num_classes: 10
preprocessing:
- name: to_tensor
loss:
  name: cross_entropy
optimizer:
  name: adam
  lr: 0.01
  weight_decay: 0.0001
  scheduler: step
  step_size: 10
  gamma: 0.1
training:
  epochs: 10
  batch_size: 256
  num_workers: 4
  seed: 42
  device: cuda
scheduler:
  name: reduce_on_plateau
  patience: 2
  factor: 0.1
metrics:
- name: f1_score
  average: weighted
callbacks:
- name: checkpoint
  dirpath: checkpoints/
  monitor: loss
  mode: min
experiment_id: resnet18_bef64961



Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Modelo
model = get_model(config.model).to(config.training.device)

# Loss, Optimizer y Scheduler
criterion = get_loss(config.loss)
optimizer= get_optimizer(config.optimizer, model.parameters())
scheduler = get_scheduler(config.scheduler, optimizer)
callbacks = get_callbacks(config.callbacks)
metrics = get_metrics(config.metrics)

In [ ]:
# Entrenador
trainer = BaseTrainer(model, criterion, optimizer, scheduler, config, logger, callbacks, metrics)
trainer.train(train_loader, val_loader)

2025-05-06 01:54:37,886 - {'Epoch': 1, 'Train_loss': '2.1758', 'Val_loss': '2.1698', 'LR': '1.00e-02', 'val_f1': '0.3438'}
2025-05-06 01:56:12,232 - {'Epoch': 2, 'Train_loss': '1.7166', 'Val_loss': '1.5303', 'LR': '1.00e-02', 'val_f1': '0.2188'}
2025-05-06 01:57:47,005 - {'Epoch': 3, 'Train_loss': '1.3069', 'Val_loss': '1.4370', 'LR': '1.00e-02', 'val_f1': '0.4321'}
2025-05-06 01:59:20,857 - {'Epoch': 4, 'Train_loss': '1.0649', 'Val_loss': '1.0987', 'LR': '1.00e-02', 'val_f1': '0.5677'}
2025-05-06 02:00:55,331 - {'Epoch': 5, 'Train_loss': '0.9229', 'Val_loss': '1.0651', 'LR': '1.00e-02', 'val_f1': '0.5646'}
2025-05-06 02:02:30,266 - {'Epoch': 6, 'Train_loss': '0.8410', 'Val_loss': '1.1195', 'LR': '1.00e-02', 'val_f1': '0.5458'}
2025-05-06 02:04:04,333 - {'Epoch': 7, 'Train_loss': '0.7771', 'Val_loss': '0.9895', 'LR': '1.00e-02', 'val_f1': '0.6821'}
2025-05-06 02:05:39,063 - {'Epoch': 8, 'Train_loss': '0.7372', 'Val_loss': '1.0875', 'LR': '1.00e-02', 'val_f1': '0.8732'}
2025-05-06 02:07

In [ ]:
len(trainer.optimizer.param_groups[0])

In [ ]:
# cargar el mejor checkpoint sobre validación en el modelo, cuya referencia ya se encuentra en el trainer
load_checkpoint(model, config)

# Evaluar métricas en el conjunto de validación
val_metrics = trainer.run_epoch(val_loader, training=False)

# Evaluar métricas en el conjunto de test

c:\Users\inaki.campo\Desktop\sw-backend\test\Experimentos Generación\utils\load_checkpoint.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f"chec

<All keys matched successfully>